In [1]:
#To remove all variables from the namespace
%reset -f

#Creating a log file to record the commands and outputs
%logstop
%logstart -t -o "E:/Python Clinical Course/Table3 log.txt"

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : E:/Python Clinical Course/Table3 log.txt
Mode           : backup
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


In [2]:
import pandas as pd
import pyreadstat
import os
from docx import Document
from docx.shared import Pt, Inches
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.oxml.ns import qn
from docx.oxml import OxmlElement

adam_path = r"E:\Python Clinical Course\ADAM datasets\ADaM Datasets"

In [3]:
adam_datasets = {}

for file in os.listdir(adam_path):
    if file.endswith(".sas7bdat"):
        dataset_name = file.replace(".sas7bdat", "")
        file_path = os.path.join(adam_path, file)
        df, meta = pyreadstat.read_sas7bdat(file_path)
        adam_datasets[dataset_name] = df

In [4]:
adsl = adam_datasets.get("adsl")

In [5]:
# Filter: Safety population with treatment
adsl_filt = adsl[(adsl['TRT01A'].notna()) & (adsl['TRT01A'] != '') & (adsl['SAFFL'] == 'Y') & (adsl['AGE'].notna())]

# Create ALL group
adsl_all = adsl_filt.copy()
adsl_all['TRT01A'] = 'ALL'
adsl_all['TRT01AN'] = 3

adsl_combined = pd.concat([adsl_filt, adsl_all], ignore_index=True)


In [6]:

# Denominator (N)
denom = adsl_combined.groupby(['TRT01AN', 'TRT01A'])['USUBJID'].nunique().reset_index(name='DENOM')
n_dict = denom.set_index('TRT01AN')['DENOM'].to_dict()
N1, N2, N3 = n_dict.get(1, 0), n_dict.get(2, 0), n_dict.get(3, 0)


In [7]:

# Summary stats
summary = adsl_combined.groupby(['TRT01AN', 'TRT01A'])['AGE'].agg(
    N='count',
    Mean='mean',
    Median='median',
    SD='std',
    Min='min',
    Max='max'
).round(2).reset_index()

# Format as string
summary['N'] = summary['N'].astype(int).astype(str)
summary['Mean'] = summary['Mean'].map('{:.1f}'.format)
summary['Median'] = summary['Median'].map('{:.1f}'.format)
summary['SD'] = summary['SD'].map('{:.2f}'.format)
summary['Min'] = summary['Min'].map('{:.0f}'.format)
summary['Max'] = summary['Max'].map('{:.0f}'.format)

# Melt for transpose
melted = summary.melt(id_vars=['TRT01AN'], value_vars=['N', 'Mean', 'SD', 'Median', 'Min', 'Max'], var_name='stat', value_name='value')


In [15]:

# Pivot
pivot = melted.pivot(index='stat', columns='TRT01AN', values='value').reset_index()

# Add label + ordering
order_map = {'N': 1, 'Mean': 2, 'SD': 3, 'Median': 4, 'Min': 5, 'Max': 6}
pivot['ORD'] = pivot['stat'].map(order_map)
pivot = pivot.sort_values('ORD')
pivot = pd.concat([pd.DataFrame({'stat': ['Age (Years)'], 'ORD': [0]}), pivot], ignore_index=True).fillna('  0')

# Rename for reporting
final = pivot.rename(columns={
    'stat': 'CATEGORY',
    1: f'DRUG A \n (N={N1})',
    2: f'DRUG B \n (N={N2})',
    3: f'ALL \n (N={N3})'
})

final=final.drop(columns='ORD')


In [16]:

# ---------------- Word Export ----------------

doc = Document()

# Margins
for section in doc.sections:
    section.top_margin = Inches(1)
    section.bottom_margin = Inches(1)
    section.left_margin = Inches(1)
    section.right_margin = Inches(1)

# Font style
style = doc.styles['Normal']
style.font.name = 'Courier New'
style.font.size = Pt(9)
style.element.rPr.rFonts.set(qn('w:eastAsia'), 'Courier New')

# Titles
doc.add_paragraph("COVID-19 AA").alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
doc.add_paragraph("Protocol: 043").alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
doc.add_paragraph("Table 14.1.3  Subject Demographics - Age (Safety Population)").alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

# Table
cols = final.columns.tolist()
table = doc.add_table(rows=1, cols=len(cols))
table.style = 'Light Shading Accent 1'
hdr = table.rows[0].cells
for i, col in enumerate(cols):
    hdr[i].text = col

for _, row in final.iterrows():
    cells = table.add_row().cells
    for i, val in enumerate(row):
        cells[i].text = str(val)

# Line
#doc.add_paragraph("_____________________________________________________________________")

# Footnote
doc.add_paragraph(r"E:TAB3_1.SAS").alignment = WD_PARAGRAPH_ALIGNMENT.LEFT


In [17]:

# Save
doc.save(r"E:\Python Clinical Course\TLF\output\t_14_1_3.docx")
